# Projeto de Classificador Automático de Sentimento

Baseado no metodo Naive-Bayes, ao longo deste projeto desenvolvemos um classificador de Tweet que opera entre 5 categorias: Muito Relevante, Relevante, Irrelevante, Muito Irrelevante e Memes.<br><br/>
Veja a seguir como fizemos isso e nossas conclusões

___
## Informações do Projeto

Grupo: Sabrina Machado e Phelipe Müller<br /><br />
GitHub: https://github.com/PhelipeMuller/Classificador-Bayes

Conta no Twitter:***@sabrina49834572***

___

## Preparando o ambiente

Importando as Bibliotecas que serão utilizadas.

In [6]:
%%capture
import math
import os.path
import pandas as pd
import json
from random import shuffle
from IPython.display import clear_output

___
## Classificando as Mensagens

Classificamos varias mensagens vindas do Twitter para usar em nosso Classificador e as separamos em dois grupo, treino e teste

___
## Montando o Classificador Naive-Bayes

Pegamos os Tweets dos arquivos de Excel, já classificados

In [7]:
produto = 'RedBull2'
n = 500
t = 300

readerTrain = pd.read_excel('{0}.xlsx'.format(produto))
readerTest = pd.read_excel('{0}.xlsx'.format(produto),sheetname = 'Teste')
train = []
test = []

for i in range(0,t):
    train.append([readerTrain['Treinamento'][i], readerTrain['Util'][i]])

for i in range(0,n-t):
    test.append([readerTest['Teste'][i],readerTest['Util'][i]])
    

Criamos duas funções que vamos utilizar ao longo do projeto

In [8]:
def multiply(n):
    total = 1
    for i in range(0, len(n)):
        total *= n[i]
    return total

def Maior(nlist):
    ValueMaior = nlist[0]
    IndexMaior = 0
    for i in range(len(nlist)):
        if nlist[i] > ValueMaior: 
            ValueMaior = nlist[i]
            IndexMaior = i
    return IndexMaior

E uma Classe com os seguintes metodos:
train: Que recebe uma lista de Tweets ja analisados e treina nosso classificador
analyze: Que baseado no treinamento 

In [9]:
class NaiveBayes:
    def __init__(self):
        self.trained = False
        self.Freq0 = 0
        self.Freq1 = 0
        self.Freq2 = 0
        self.Freq3 = 0
        self.Freq4 = 0
        self.WordFreqList = [["redbull",0,0,0,0,0]]
                            #[Palavra, Freq0, Freq1, Freq2, Freq3, Freq4]
        self.TotalWord = [1,0,0,0,0,0]
                            #Palavra, Freq0, Freq1, Freq2, Freq3, Freq4
        self.CatProb=[0,0,0,0,0]
        
    def train(self,train):
        for tweet in train:
            if tweet[1] == 0: self.Freq0 +=1 #1 = Coluna Util
            elif tweet[1] == 1: self.Freq1 +=1 #1 = Coluna Util
            elif tweet[1] == 2: self.Freq2 +=1 #1 = Coluna Util
            elif tweet[1] == 3: self.Freq3 +=1 #1 = Coluna Util
            elif tweet[1] == 4: self.Freq4 +=1
            tweet[0] = self.normalizeTweet(tweet[0])#0 = Coluna Treinamento
            WordList = tweet[0].split(' ') #0 = Coluna Treinamento
            for word in WordList:
                onList = False
                for freq in self.WordFreqList:
                    if word == freq[0]:
                        onList = True
                        if tweet[1] == 0: 
                            freq[1]+=1 #1 = Coluna Util
                            self.TotalWord[1]+=1
                        elif tweet[1] == 1: 
                            freq[2]+=1 #1 = Coluna Util
                            self.TotalWord[2]+=1
                        elif tweet[1] == 2: 
                            freq[3]+=1 #1 = Coluna Util
                            self.TotalWord[3]+=1
                        elif tweet[1] == 3: 
                            freq[4]+=1 #1 = Coluna Util
                            self.TotalWord[4]+=1
                        elif tweet[1] == 4: 
                            freq[5]+=1
                            self.TotalWord[5]+=1    
                if not onList:
                    self.TotalWord[0]+=1
                    if tweet[1] == 0: self.WordFreqList.append([word,1,0,0,0,0]) #1 - Coluna Util
                    elif tweet[1] == 1: self.WordFreqList.append([word,0,1,0,0,0]) #1 - Coluna Util
                    elif tweet[1] == 2: self.WordFreqList.append([word,0,0,1,0,0]) #1 - Coluna Util
                    elif tweet[1] == 3: self.WordFreqList.append([word,0,0,0,1,0]) #1 - Coluna Util
                    elif tweet[1] == 4: self.WordFreqList.append([word,0,0,0,0,1])
        self.trained = True
        self.CatProb[0] = self.Freq0/300
        self.CatProb[1] = self.Freq1/300
        self.CatProb[2] = self.Freq2/300
        self.CatProb[3] = self.Freq3/300
        self.CatProb[4] = self.Freq4/300
        return 0
    
    def test(self, test):
        if self.trained:
            acertoTotal = 0 
            erroTotal = 0
            acerto0 = 0
            erro0 = 0
            acerto1 = 0
            erro1 = 0
            acerto2 = 0
            erro2 = 0
            acerto3 = 0
            erro3 = 0
            acerto4 = 0
            erro4 = 0
            for tweet in test:
                if self.analyze(tweet)==tweet[1]:
                    acertoTotal+=1
                    if tweet[1] == 0: acerto0 +=1
                    elif tweet[1] == 1: acerto1 +=1
                    elif tweet[1] == 2: acerto2 +=1
                    elif tweet[1] == 3: acerto3 +=1
                    elif tweet[1] == 4: acerto4 +=1
                else:
                    erroTotal +=1
                    if tweet[1] == 0: erro0 +=1
                    elif tweet[1] == 1: erro1 +=1
                    elif tweet[1] == 2: erro2 +=1
                    elif tweet[1] == 3: erro3 +=1
                    elif tweet[1] == 4: erro4 +=1

            print("Muito Irrelevante Falso: " + str((erro0 /(n-t))*100)+"%")
            print("Muito Irrelevante Verdadeiro: " + str((acerto0 /(n-t))*100)+"%")
            print()
            print("Irrelevante Falso: " + str((erro1 /(n-t))*100)+"%")
            print("Irrelevante Verdadeiro: " + str((acerto1 /(n-t))*100)+"%")
            print()
            print("Meme Falso: " + str((erro2 /(n-t))*100)+"%")
            print("Meme Verdadeiro: " + str((acerto2 /(n-t))*100)+"%")
            print()
            print("Relevante Falso: " + str((erro3 /(n-t))*100)+"%")
            print("Relevante Verdadeiro: " + str((acerto3 /(n-t))*100)+"%")
            print()
            print("Muito Relevante Falso: " + str((erro4 /(n-t))*100)+"%")
            print("Muito Relevante Verdadeiro: " + str((acerto4 /(n-t))*100)+"%")

            print()
            print()
            print()

            print("Acerto: " + str(((acerto0 + acerto1 + acerto2 + acerto3 + acerto4)/(n-t))*100)+"%")
            print("Erro: "  + str(((erro0 + erro1 + erro2 + erro3 + erro4)/(n-t))*100)+"%")

            print()
            print()
            print()

            print("Total: " + str(((erro0 + erro1 + erro2 + erro3 + erro4 + acerto0 + acerto1 + acerto2 + acerto3 + acerto4)/(n-t))*100)+"%")
        else: print("Você precisa treinar seu classificador antes de tentar analisar algo")
    
    def analyze(self,tweet):
        
        tweet[0] = self.normalizeTweet(tweet[0])#0 = Coluna Treinamento
        WordList = tweet[0].split(' ') #0 = Coluna Treinamento
        relevancia = [] #acho que nao ta sendo usado
        probWord = []
        for word in WordList:
            onList = False
            for freq in self.WordFreqList:
                if word == freq[0]:
                    onList = True
                    prob0 = (freq[1]+1)/(self.TotalWord[1]+self.TotalWord[0])
                    prob1 = (freq[2]+1)/(self.TotalWord[2]+self.TotalWord[0])
                    prob2 = (freq[3]+1)/(self.TotalWord[3]+self.TotalWord[0])
                    prob3 = (freq[4]+1)/(self.TotalWord[4]+self.TotalWord[0])
                    prob4 = (freq[5]+1)/(self.TotalWord[5]+self.TotalWord[0])
                    probTotal = (freq[1]+freq[2]+freq[3]+freq[4]+freq[5] +1)/self.TotalWord[0]
                    probWord.append([probTotal, prob0, prob1, prob2, prob3, prob4])
                    
            if not onList:
                probWord.append([1/self.TotalWord[0], 1/(self.TotalWord[1]+self.TotalWord[0]+1), 1/(self.TotalWord[2]+self.TotalWord[0]+1), 1/(self.TotalWord[3]+self.TotalWord[0]+1), 1/(self.TotalWord[4]+self.TotalWord[0]+1), 1/(self.TotalWord[5]+self.TotalWord[0]+1)])
        try:pW1 = multiply(probWord[:][1])*self.CatProb[0]
        except:pW1 = 0
        try:pW2 = multiply(probWord[:][2])*self.CatProb[1]
        except:pW2 = 0
        try:pW3 = multiply(probWord[:][3])*self.CatProb[2]
        except:pW3 = 0
        try:pW4 = multiply(probWord[:][4])*self.CatProb[3]
        except:pW4 = 0
        try:pW5 = multiply(probWord[:][5])*self.CatProb[4]
        except:pW5 = 0
        
        probTweet = [pW1,pW2,pW3,pW4,pW5]
        return Maior(probTweet)  
 
    def normalizeTweet(self,tweet):
        tweet.lower()
        tweet = tweet.replace(',',' ')
        tweet = tweet.replace('!',' ')
        tweet = tweet.replace('$',' ')
        tweet = tweet.replace('%',' ')
        tweet = tweet.replace('&',' ')
        tweet = tweet.replace('*',' ')
        tweet = tweet.replace('(',' ')
        tweet = tweet.replace(')',' ')
        tweet = tweet.replace('-',' ')
        tweet = tweet.replace('+',' ')
        tweet = tweet.replace('=',' ')
        tweet = tweet.replace('_',' ')
        tweet = tweet.replace('.',' ')
        tweet = tweet.replace(':',' ')
        tweet = tweet.replace(';',' ')
        tweet = tweet.replace('?',' ')
        tweet = tweet.replace('/',' ')
        tweet = tweet.replace('\n',' ')
        for i in range(0,4): tweet = tweet.replace('  ',' ')
        return tweet
                              
    def show(self):
        if self.trained: print("Treinado com " + str(len(self.WordFreqList)) + " Palavras")
        else: print("Precisa ser treinado")
    
    def showAll(self):
        if self.trained:
            print(self.WordFreqList)
            print()
            print("Treinado com " + str(len(self.WordFreqList)) + " Palavras")
        else: print("Precisa ser treinado")
        


In [10]:
Classificador = NaiveBayes()
Classificador.train(train)
clear_output()

___
## Verificando a performance

In [11]:
Classificador.test(test)

Muito Irrelevante Falso: 13.5%
Muito Irrelevante Verdadeiro: 2.0%

Irrelevante Falso: 16.5%
Irrelevante Verdadeiro: 2.5%

Meme Falso: 52.5%
Meme Verdadeiro: 3.5000000000000004%

Relevante Falso: 8.5%
Relevante Verdadeiro: 1.0%

Muito Relevante Falso: 0.0%
Muito Relevante Verdadeiro: 0.0%



Acerto: 9.0%
Erro: 91.0%



Total: 100.0%


___
## Concluindo

Opcionalmente: 
* Discorrer por que não posso alimentar minha base de Treinamento automaticamente usando o próprio classificador, aplicado a novos tweets.[ ]
* Propor diferentes cenários de uso para o classificador Naive-Bayes. Cenários sem intersecção com este projeto.[ ]


O nosso Classificador Naive-Bayes analisa propabilidade de um tweet ser relevante, irrelevante ou um meme/piada baseado nas palavras usadas nele. Para conhecer a probalidade de relevancia de uma palavra fornecemos um primeiro grupo de tweet para treina-lo.
Tivemos um percentual de analises corretas extremamente baixas, compreendemos o motivo disso atraves de nosso Classificador Naive Bayes de apenas 2 Categorias, que pode ser encontrado no arquivo de apoio no gitHub (ArquivoDeApoio.ipynb), assim como lá o que o classificador fez, foi sempre ter uma chance maior de julgar que um determinado Tweet pertence ao grupo com mais amostras no grupo de treino. No Classificador com apenas 2 Categorias isso não se mostrou tão problematico, já que alem da categoria irrelevante possuir o maior numero de Tweet em relação a outra categoria, ele tambem possuia a maioria dos Tweet da amostra. Porem ao acinonarmos mais categorias a categoria 2(Memes) passa a ser a categoria com maior numero de Tweets em relação as outras categorias, mas não mais a maioria dos Tweets da amostra. Fazendo assim com que a maioria dos Tweets analisados fossem classificados como Memes.

Como não existe uma analise completa do tweet, nosso clasificador não reconhece contexto, sarcasmo, duplas negações ou qualquer outro aspecto da frase que não seja a probalidade de suas palavras tornararem o proprio tweet relevante, isso faz com que o classificador cometa diverssos erros, já que podemos usar certas palavras em diversos contextos com divesos significados diferentes.

___
Em nossas proximas iterações, para podermos aprimorar nosso Classificador e sem alterar o tipo dele (Naive-Bayes), vamos por exemplo:
    
* Aprimorar a qualidade do banco de Treino<br><br />
* Ao se deparar com um Tweet, por exemplo: "Eu gosto de beber Redbull sextas a noite", alem de analisar e registrar cada uma das palavras nossa classificador passará a analisar tambem fragmentos do tweet, como: "Eu gosto de", "beber Redbull" e "sextas a noite"<br><br />
* Passar a não registrar artigos de ligação, como: "de", "do", "dos", "da", "das", etc.<br><br />
* Fornecer ao Classificador um grupo de regras para transformar abreviações em sua versão completa e corrigir erros de escrita, para que dessa forma não registre duas palavras iguais como duas diferentes, tal como: "vc" e "você", aumentando a qualidade do treinamento e de analise do Classificador.<br><br />
* Desenvolver uma tecnica de associação para que as categorias ligadas como: Relevante, Irrelevante, Muito Relevante e Muito Irrelevante, atuem como uma sequencia de uma da outra e baseado em um sistema de pontuação o Classificador possa decidir em qual categoria o Tweet esta<br><br />
* Combinar a analise Naive-Bayes com a analise de uma Inteligencia Artificial 100% programada, baseada as regras que nós mesmos utilizamos para classificar os Tweets, tal como: Se não ao menos fala do produto é Muito Irrelevante, se fala do produto mas é um reTweet é Irrelevante mesmo que o Tweet Inicial fosse Relevante